In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch
from tqdm.auto import tqdm
import collections

from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering
from transformers import TrainingArguments
from transformers import Trainer
import evaluate

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
MODEL_NAME = 'distilbert-base-uncased'

MAX_LENGTH = 384

STRIDE = 128

In [ ]:
DATASET_NAME = 'squad_v2'
raw_datasets = load_dataset(DATASET_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [ ]:
raw_datasets['train']['question']

['When did Beyonce start becoming popular?',
 'What areas did Beyonce compete in when she was growing up?',
 "When did Beyonce leave Destiny's Child and become a solo singer?",
 'In what city and state did Beyonce  grow up? ',
 'In which decade did Beyonce become famous?',
 'In what R&B group was she the lead singer?',
 'What album made her a worldwide known artist?',
 "Who managed the Destiny's Child group?",
 'When did Beyoncé rise to fame?',
 "What role did Beyoncé have in Destiny's Child?",
 'What was the first album Beyoncé released as a solo artist?',
 'When did Beyoncé release Dangerously in Love?',
 'How many Grammy awards did Beyoncé win for her first solo album?',
 "What was Beyoncé's role in Destiny's Child?",
 "What was the name of Beyoncé's first solo album?",
 'After her second solo album, what other entertainment venture did Beyonce explore?',
 'Which artist did Beyonce marry?',
 'To set the record for Grammys, how many did Beyonce win?',
 'For what movie did Beyonce rec

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess_training_examples(examples):
  questions = [q.strip() for q in examples['question']]

  inputs = tokenizer(
      questions,
      examples['context'],
      max_length=MAX_LENGTH,
      truncation='only_second',
      stride=STRIDE,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding='max_length'
  )

  offset_mapping = inputs.pop('offset_mapping')

  sample_map = inputs.pop('overflow_to_sample_mapping')

  answers = examples['answers']

  start_positions = []
  end_positions = []

  for idx, offset in enumerate(offset_mapping):
    sample_idx = sample_map[idx]

    sequence_ids = inputs.sequence_ids(idx)

    idx = 0
    while sequence_ids[idx] != 1:
      idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
      idx += 1
    context_end = idx - 1

    answer = answers[sample_idx]

    if len(answer['text']) == 0:
      start_positions.append(0)
      end_positions.append(0)
    else:
      start_char = answer['answer_start'][0]
      end_char = answer['answer_start'][0] + len(answer['text'][0])

      if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
      else:
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
          idx += 1
        start_positions.append(idx - 1)
        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
          idx -= 1
        end_positions.append(idx + 1)

  inputs['start_positions'] = start_positions
  inputs['end_positions'] = end_positions

  return inputs

In [ ]:
train_dataset = raw_datasets['train'].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets['train'].column_names,
)

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

In [ ]:
len(raw_datasets['train']), len(train_dataset)

(130319, 131754)

In [ ]:
def preprocess_validation_examples(examples):
  questions = [q.strip() for q in examples['question']]

  inputs = tokenizer(
      questions,
      examples['context'],
      max_length=MAX_LENGTH,
      truncation='only_second',
      stride=STRIDE,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding='max_length'
  )

  sample_map = inputs.pop('overflow_to_sample_mapping')
  example_ids = []

  for idx in range(len(inputs['input_ids'])):
    sample_idx = sample_map[idx]
    example_ids.append(examples['id'][sample_idx])

    sequence_ids = inputs.sequence_ids(idx)
    offset = inputs['offset_mapping'][idx]

    inputs['offset_mapping'][idx] = [
        o if sequence_ids[k] == 1 else None \
        for k, o in enumerate(offset)
    ]

  inputs['example_id'] = example_ids

  return inputs

In [ ]:
validation_dataset = raw_datasets['validation'].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets['validation'].column_names,
)

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
args = TrainingArguments(
    output_dir='distilbert-finetuned-squadv2',
    evaluation_strategy='no',
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=True
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from huggingface_hub import login
login(token="HUGGGINGFACE_TOKEN")

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)

<ipython-input-17-63f3f3225169>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tranhuysuxt (huytranduck) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,3.120800
1000,2.229100
1500,1.879000
2000,1.759900
2500,1.661100
3000,1.614100
3500,1.536700
4000,1.514000
4500,1.484700
5000,1.489500


TrainOutput(global_step=49410, training_loss=1.0033866896221113, metrics={'train_runtime': 4845.9456, 'train_samples_per_second': 81.566, 'train_steps_per_second': 10.196, 'total_flos': 3.873165421863629e+16, 'train_loss': 1.0033866896221113, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub(commit_message='Traning complete')

events.out.tfevents.1741761483.9b11bfcd35cb.558.0:   0%|          | 0.00/26.3k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/huytranduck/distilbert-finetuned-squadv2/commit/82005a3e6e6c3f6106fd67b039cbe48c61d9a048', commit_message='Traning complete', commit_description='', oid='82005a3e6e6c3f6106fd67b039cbe48c61d9a048', pr_url=None, repo_url=RepoUrl('https://huggingface.co/huytranduck/distilbert-finetuned-squadv2', endpoint='https://huggingface.co', repo_type='model', repo_id='huytranduck/distilbert-finetuned-squadv2'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline

PIPELINE_NAME = 'question-answering'
MODEL_NAME = 'huytranduck/distilbert-finetuned-squadv2'
pipe = pipeline(PIPELINE_NAME, model=MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/561 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
INPUT_QUESTION = 'What is my name?'
INPUT_CONTEXT = 'Hello! My name is Huy, and I am currently a fourth-year student at the Posts and Telecommunications Institute of Technology (PTIT).'
pipe(question=INPUT_QUESTION, context=INPUT_CONTEXT)

{'score': 0.9368132948875427, 'start': 18, 'end': 21, 'answer': 'Huy'}

In [ ]:
pipe2 = pipeline(PIPELINE_NAME, model='distilbert-base-uncased')
pipe2(INPUT_QUESTION, INPUT_CONTEXT)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


{'score': 0.0020619293209165335, 'start': 129, 'end': 131, 'answer': ').'}